In [1]:
using PauliPropagation
using Random
using Optim
using Plots
Random.seed!(43)
using ReverseDiff: GradientTape, gradient!, compile, gradient
using LinearAlgebra
using StatsBase 
using GLM
using DataFrames
using CSV

# CDR for quantum dynamics
- TFIM Hamiltonian (constants not site-dependent) evolving with TDSE
- discretize the evolution steps and use first-order Trotter decomposition 

### Idea:
- DONE: Run "exact" evolution for small trotterized circuit
- Create near-Clifford circuits (replace some gates in Trotterized gate by close Cliffords - no VQA here so this is much simpler, find closest Clifford to the RZZ and to the RX (only have 2 angles in total), replace a portion randomly (no MCMC), keep the parameter of N = non-Cliffords const. st. system remains cl. scalable.
- create MWE of CDR within quantum dynamics
- which gates ones we replace has influence on the accuracy of the expectation value (principle of causal light cone as seen in vnCDR paper Piotr, is this automatically respected within Heisenberg picture? (backprop observable)), so we can aim to replace only gates that contribute to an expectation value (works only for single-qubit / local observables)
- another idea is to try the perturbation approach (all non-CLiffords) 



### Exact evolution of a small trotterized circuit (see CPDR p.7)

In [2]:
### Trotter circuit in PP ###

# function tfitrottercircuit(nqubits::Integer, nlayers::Integer; topology=nothing, start_with_ZZ=true)
#     circuit::Vector{Gate} = []

#     if isnothing(topology)
#         topology = bricklayertopology(nqubits)
#     end

#     # the function after this expects a circuit with at least one layer and will always append something
#     if nlayers == 0
#         return circuit
#     end

#     if start_with_ZZ
#         rzzlayer!(circuit, topology)
#     end

#     for _ in 1:nlayers-1
#         rxlayer!(circuit, nqubits)
#         rzzlayer!(circuit, topology)
#     end

#     rxlayer!(circuit, nqubits)

#     if !start_with_ZZ
#         rzzlayer!(circuit, topology)
#     end

#     return circuit
# end

In [3]:
T = 1.0
h = 1.0 #abs(h)<abs(J) for ordered phase (interaction term dominates)
J = 5.0 #H = -J ZZ + h X, so J>0 ferromagnetic, J<0 antiferromagnetic.
steps = 10 # number of steps is number of layers of Trotter circuit
sigma_h = 2*T*h/steps 
sigma_J = -2*T*J/steps

nq = 4
#topology = ibmeagletopology (127 qubits)-to use heavyhex with less qubits, see TP4 notebook
circuit = tfitrottercircuit(nq,steps,topology=nothing)
nparams = countparameters(circuit)
println("Number of parameters: ", nparams)

sigma_h_indices = getparameterindices(circuit, PauliRotation, [:X])
sigma_J_indices = getparameterindices(circuit, PauliRotation, [:Z,:Z])
println("Sigma h indices: ", length(sigma_h_indices))
println("Sigma J indices: ", length(sigma_J_indices))
println("Sigma J indices: ", sigma_J_indices)
thetas = zeros(nparams)
thetas[sigma_h_indices] .= sigma_h
thetas[sigma_J_indices] .= sigma_J;

Number of parameters: 70
Sigma h indices: 40
Sigma J indices: 30
Sigma J indices: [1, 2, 3, 8, 9, 10, 15, 16, 17, 22, 23, 24, 29, 30, 31, 36, 37, 38, 43, 44, 45, 50, 51, 52, 57, 58, 59, 64, 65, 66]


In [4]:
#define observable: magnetization in z direction
magnetization = PauliSum(nq)
for i in 1:nq
    add!(magnetization,:Z,i)
end
magnetization = magnetization/nq

PauliSum(nqubits: 4, 4 Pauli terms:
 0.25 * IZII
 0.25 * ZIII
 0.25 * IIIZ
 0.25 * IIZI
)

In [5]:
@time psum = propagate(circuit, magnetization, thetas)

  0.357734 seconds (347.17 k allocations: 17.374 MiB, 99.87% compilation time)


PauliSum(nqubits: 4, 128 Pauli terms:
 -0.00052564 * YZYX
 0.02338 * YXII
 -0.0080699 * ZIIX
 -0.0094663 * ZYXZ
 0.00027512 * XYIX
 -0.0018116 * YXZY
 -0.0096923 * XZZZ
 -0.054098 * IIZX
 -0.00052564 * XYZY
 0.0045791 * ZYIY
 -0.054098 * XZII
 0.0014803 * YXXX
 0.0048063 * IYYY
 0.038127 * ZZIZ
 -0.0029081 * YIZZ
 -0.00078975 * ZXZY
 -0.00092879 * IXZI
 -0.0014466 * ZIZY
 0.085887 * IZII
 -0.064918 * ZXXI
  ⋮)

In [6]:
overlapwithzero(psum) #should be close to 1 (we are in the ferromagnetic phase) 

0.9473863474250936

### Training set generation

In [7]:
# select which and how many non-Cliffords to replace
non_cliffs = 30 
cliffs= nparams-non_cliffs
ratio = length(sigma_J_indices)/length(sigma_h_indices)
num_h = Int(round(ratio*cliffs))
num_J = Int(round((1-ratio)*cliffs))
Random.shuffle!(sigma_h_indices)
Random.shuffle!(sigma_J_indices)
selected_indices_h = sigma_h_indices[1:num_h]
selected_indices_J = sigma_J_indices[1:num_J];   

# find the closest Clifford gate to RZZ and RX respectively

# replace the non-Cliffords with the closest Clifford gates


10-element Vector{Int64}:
 51
 10
 58
 65
 43
 38
 24
 57
  8
 36